In [1]:
cd ..

/home/alberto/PycharmProjects/edc_mo_pdac_val


/home/alberto/anaconda3/envs/edc_mo_pdac_val/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


# Multi-omics stratification on PDAC patients

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import NMF
from sklearn.preprocessing import StandardScaler
import optuna
import time
import dill
import shutil
from optuna.samplers import TPESampler

from src import settings
from src.optimization import Optimization


In [3]:
data_folder = "data"
processed_data_folder = "processed"
processed_data_path = os.path.join(data_folder, processed_data_folder)

## Load dataset

In [4]:
files = os.listdir(processed_data_path)
files = list(filter(lambda x: os.path.isfile(os.path.join(processed_data_path, x)), files))
file = -1

In [5]:
Xs = [StandardScaler().set_output(transform="pandas").fit_transform(pd.read_csv(os.path.join(processed_data_path, filename), index_col=0)) for filename in files]
samples = Xs[0].index

In [6]:
file += 1
filename = files[file]
data = pd.read_csv(os.path.join(processed_data_path, filename), index_col=0)
print(f"{filename}.shape", data.shape)
data.head()

plasma_data.csv.shape (39, 663)


,plasma_protein_A0M8Q6_LAC7,plasma_protein_B9A064_IGLL5,plasma_protein_O14791_APOL1,plasma_protein_O14980_XPO1,plasma_protein_O15212_PFD6,plasma_protein_O15230_LAMA5,plasma_protein_O43866_CD5L,plasma_protein_O75369_FLNB,plasma_protein_O75390_CISY,plasma_protein_O75882_ATRN,...,plasma_lipid_fatty_acid_comp_TAG(FA16:0),plasma_lipid_fatty_acid_comp_TAG(FA16:1),plasma_lipid_fatty_acid_comp_TAG(FA17:0),plasma_lipid_fatty_acid_comp_TAG(FA18:0),plasma_lipid_fatty_acid_comp_TAG(FA18:1),plasma_lipid_fatty_acid_comp_TAG(FA18:2),plasma_lipid_fatty_acid_comp_TAG(FA18:3),plasma_lipid_fatty_acid_comp_TAG(FA20:4),plasma_lipid_fatty_acid_comp_TAG(FA20:5),plasma_lipid_fatty_acid_comp_TAG(FA22:6)
GI-19-025,5.020191,5.451051,5.305104,5.781695,4.622499,4.318202,5.379278,4.856250,5.164457,3.899591,...,0.0842,0.2654,1.4852,0.3907,27.4427,2.6384,21.3749,0.0903,2.3547,0.7212
GI-17-537,5.215109,5.590096,5.403843,5.535984,4.622499,4.318202,5.672487,4.673977,3.828509,3.845567,...,0.3447,0.5042,2.8104,0.5411,26.7298,3.9978,21.4354,0.0946,2.2662,0.4359
GI-19-002,5.045229,5.370674,5.106038,5.457131,4.932033,4.953377,5.713943,4.856250,4.539673,3.723098,...,0.1580,0.4536,3.0685,0.4594,22.6095,4.4593,22.8729,0.1054,1.8585,1.3486
GI-16-575,5.151848,5.656317,5.294460,5.517482,4.969732,4.198990,5.371976,4.575006,4.539673,3.359389,...,21.0218,2.3462,0.4330,3.6900,39.5313,24.9905,1.9591,1.3469,0.4768,1.6169
GI-14-974,4.991846,5.427921,5.122799,5.303777,4.622499,4.400274,5.260241,4.812790,4.539673,3.706972,...,26.1365,3.0751,0.4406,2.6781,30.2909,27.8647,2.3295,2.4099,0.2761,1.3799


In [7]:
file += 1
filename = files[file]
data = pd.read_csv(os.path.join(processed_data_path, filename), index_col=0)
print(f"{filename}.shape", data.shape)
data.head()

tissue_data.csv.shape (39, 1130)


,tissue_protein_A0A075B6S5_KV127,tissue_protein_A0A0C4DH35_HV335,tissue_protein_A0A0C4DH36_HV338,tissue_protein_A0AV96_RBM47,tissue_protein_A0FGR8_ESYT2,tissue_protein_A0MZ66_SHOT1,tissue_protein_A1X283_SPD2B,tissue_protein_A6NMY6_AXA2L,tissue_protein_A6NNZ2_TBB8B,tissue_protein_O00154_BACH,...,tissue_protein_Q9Y450_HBS1L,tissue_protein_Q9Y4E8_UBP15,tissue_protein_Q9Y5S9_RBM8A,tissue_protein_Q9Y5Y2_NUBP2,tissue_protein_Q9Y678_COPG1,tissue_protein_Q9Y6A9_SPCS1,tissue_protein_Q9Y6B6_SAR1B,tissue_protein_Q9Y6C2_EMIL1,tissue_protein_Q9Y6M9_NDUB9,tissue_protein_Q9Y6Y8_S23IP
GI-19-025,4.713994,4.233653,4.298054,3.697783,4.263094,3.460593,4.130536,3.517030,4.495384,3.881753,...,3.47681,4.612968,4.395142,3.398342,3.801523,4.395557,4.195019,4.883127,3.837916,4.260869
GI-17-537,4.713994,4.233653,4.298054,3.853952,4.366608,3.460593,3.643790,3.517030,4.495384,3.881753,...,3.47681,5.794042,4.432392,3.398342,5.252542,4.395557,3.782099,4.262695,4.683767,4.831078
GI-19-002,4.713994,4.640927,4.366604,3.697783,4.263094,3.460593,3.643790,3.517030,4.495384,3.881753,...,3.47681,4.612968,4.687704,3.398342,3.665189,4.395557,4.195019,5.625306,4.161362,4.260869
GI-16-575,5.050063,4.304040,4.293424,3.697783,4.263094,3.460593,3.643790,4.095981,4.940837,4.364585,...,3.47681,4.612968,4.395142,3.398342,5.518563,4.277215,4.381231,6.193125,4.806804,3.985681
GI-14-974,4.383966,4.039878,4.298054,3.865179,4.686643,3.460593,3.643790,4.175468,5.024638,4.694124,...,3.47681,4.612968,4.395142,3.398342,5.912920,4.394835,5.035698,5.979105,5.016900,3.781480


In [8]:
file += 1
filename = files[file]
data = pd.read_csv(os.path.join(processed_data_path, filename), index_col=0)
print(f"{filename}.shape", data.shape)
data.head()

pathology_data.csv.shape (39, 820)


,pathology_NF40_25%,pathology_NF39_0.01,pathology_NF62_75%,pathology_NF58_0.01,pathology_NF46_0.1,pathology_NF16_0.95,pathology_NF59_min,pathology_NF22_std,pathology_NF37_75%,pathology_NF20_0.05,...,pathology_NF4_50%,pathology_NF63_max,pathology_NF49_50%,pathology_NF32_0.95,pathology_NF15_0.1,pathology_NF26_0.1,pathology_NF6_25%,pathology_NF28_0.1,pathology_NF5_50%,pathology_NF3_mean
GI-19-025,18.221364,0.083023,31.048349,0.884340,16971.053546,0.919779,0.165546,0.224469,3293.252206,363.037625,...,0.806650,361.342423,0.254049,0.023666,29.383830,0.948465,383.00,848.719310,0.591029,25.556694
GI-17-537,17.700665,0.084998,39.661064,0.663660,14967.563271,0.911952,0.165851,0.254959,3411.797462,339.028141,...,0.703743,407.634914,0.257888,0.024600,32.485244,0.936644,460.00,756.448280,0.710455,28.042728
GI-19-002,17.255396,0.085569,32.000000,0.769103,16708.054344,0.908448,0.166843,0.214322,3415.990272,362.079941,...,0.791136,315.220854,0.260066,0.018552,33.917936,0.942633,390.00,898.706989,0.611640,25.718516
GI-16-575,17.291761,0.085349,33.615473,0.679883,15951.062804,0.905980,0.164000,0.229177,3411.205173,360.440383,...,0.781992,450.018046,0.259194,0.019614,33.828096,0.943830,387.00,842.917498,0.623288,25.909778
GI-14-974,18.739805,0.083579,110.367568,0.850576,15711.336968,0.934820,0.189802,0.193320,2678.358771,369.434757,...,0.745780,722.832459,0.247976,0.019650,26.712580,0.952773,339.75,804.602788,0.666186,24.707599


In [9]:
file += 1
filename = files[file]
data = pd.read_csv(os.path.join(processed_data_path, filename), index_col=0)
print(f"{filename}.shape", data.shape)
data.head()

CNV_data.csv.shape (39, 648)


,CNV_HLA-DPB1,CNV_GPC3,CNV_SOX2,CNV_XRCC3,CNV_PAK1,CNV_ACVR1B,CNV_DDR2,CNV_FANCL,CNV_CDKN1B,CNV_DYNC2H1,...,CNV_NBN,CNV_HLA-DQB2,CNV_BRCA2,CNV_RAD51C,CNV_NOTCH2,CNV_DNM2,CNV_TNFRSF9,CNV_FGF9,CNV_ECT2L,CNV_CDKN1A
GI-19-025,0.000,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
GI-17-537,0.000,0.0,0.000,0.000,0.000,0.000,0.249,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
GI-19-002,0.000,0.0,0.000,0.000,0.000,0.000,0.198,-0.285,0.000,0.000,...,0.000,0.000,0.000,0.000,0.198,0.000,-0.277,0.000,0.000,0.000
GI-16-575,-0.217,0.0,0.000,-0.007,0.121,-0.174,0.000,-0.036,-0.174,0.121,...,-0.066,-0.217,0.040,-0.092,0.000,0.208,0.000,0.040,-0.217,-0.217
GI-14-974,-0.914,0.0,0.075,0.004,0.000,0.000,-0.004,0.000,0.000,0.035,...,0.019,0.000,0.108,-0.056,-0.004,0.071,-0.004,0.108,-0.019,-0.019


In [10]:
file += 1
filename = files[file]
data = pd.read_csv(os.path.join(processed_data_path, filename), index_col=0)
print(f"{filename}.shape", data.shape)
data.head()

freebayes_data.csv.shape (39, 611)


,freebayes_SNV_HES4,freebayes_SNV_PPP1R3E,freebayes_SNV_UBA52,freebayes_SNV_FGF20,freebayes_SNV_RIT1,freebayes_SNV_IL6R,freebayes_SNV_SOX2,freebayes_SNV_SYNE1,freebayes_SNV_DOT1L,freebayes_SNV_RAD51D,...,freebayes_SNV_MAP3K1,freebayes_SNV_MIR650-MIR5571,freebayes_SNV_MAGEB1-NR0B1,freebayes_SNV_TAP2.2,freebayes_SNV_SDHD,freebayes_SNV_PMS2,freebayes_SNV_PTPRT,freebayes_SNV_HEY2,freebayes_SNV_SOX9,freebayes_SNV_RICTOR
GI-19-025,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GI-17-537,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
GI-19-002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GI-16-575,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GI-14-974,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [11]:
file += 1
filename = files[file]
data = pd.read_csv(os.path.join(processed_data_path, filename), index_col=0)
print(f"{filename}.shape", data.shape)
data.head()

rna_data.csv.shape (39, 2000)


,rna_expr_IL6ST,rna_expr_LPP,rna_expr_REG1A,rna_expr_WASF2,rna_expr_UBXN7,rna_expr_NFAT5,rna_expr_MT-ND4,rna_expr_NOTCH2,rna_expr_ARID1A,rna_expr_PLEC,...,rna_expr_NISCH,rna_expr_MFN2,rna_expr_RNF103,rna_expr_KCNK1,rna_expr_VASH1,rna_expr_TBC1D4,rna_expr_EPS15L1,rna_expr_MANBA,rna_expr_ZNF561,rna_expr_CHD8
GI-19-025,241.035075,1021.269300,13.729300,147.2934,103.106320,94.383909,14.84040,304.516089,189.958217,324.729539,...,225.176181,39.375050,95.598693,38.908040,64.202540,63.034703,50.252791,32.985904,90.178841,61.951973
GI-17-537,215.826456,478.585126,8.704680,132.7357,86.787428,78.155881,3.32408,124.369304,154.196223,479.120290,...,133.896703,35.331787,62.410831,45.372260,18.359144,19.237960,29.319210,46.584303,38.170383,68.306816
GI-19-002,305.901310,506.337486,281.889123,161.4266,114.331597,95.955602,12.47790,183.312681,221.292130,205.174571,...,165.992257,21.930622,194.303080,76.009044,42.496390,79.657814,49.634208,41.942784,93.740426,67.387638
GI-16-575,232.227397,423.957525,178.389507,172.6827,104.493370,106.009576,18.69510,302.722650,233.581237,282.710050,...,165.535513,36.941221,193.460040,61.266900,56.176480,106.721880,49.656624,43.322795,92.509717,62.555034
GI-14-974,263.865278,1120.498530,2177.204700,142.0982,95.410231,87.508449,15.89970,192.934799,260.016111,180.536180,...,137.285121,51.824260,106.984220,16.465899,40.672892,54.942205,64.974170,24.602461,95.077477,75.541299


In [12]:
new_study = False
if new_study:
    shutil.rmtree("tensorboard/", ignore_errors= True)
    date = time.strftime('%Y%m%d%H')
    optimization_study = optuna.create_study(direction="maximize", sampler=TPESampler(seed = 42, multivariate = True, n_startup_trials = 1000))
    n_trials = 1
    for file in os.listdir(settings.optimization_path):
        try:
            os.remove(os.path.join(settings.optimization_path, file))
        except IsADirectoryError:
            shutil.rmtree(os.path.join(settings.optimization_path, file), ignore_errors= True)
else:
    date = "2024050715"
    with open(os.path.join(settings.optimization_path, f'optimization_optuna_{date}.pkl'), 'rb') as file:
        optimization_study = dill.load(file)
    n_trials = 2000

In [ ]:
# %%capture --no-display

func_objective = lambda trial: Optimization().objective(trial= trial, Xs= Xs, samples= samples, num_layers_option= [1,2,1], num_units_option= [2,6, 2],
                                                        n_epochs_option= [20,100,20], lambda_option = [0.001, 1, 0.25], n_clusters_option= [2,4,1],
                                                        latent_space_option = [32, 128, 32], batch_size=32,
                                                        random_state=settings.RANDOM_STATE, n_jobs= 1)

keep_trying = True
while keep_trying:
    try:
        optimization_study = Optimization.optimize_optuna_and_save(study= optimization_study, n_trials = n_trials, date=date,
                                                                   show_progress_bar= True, folder= settings.optimization_path, func= func_objective)
        if new_study:
            keep_trying = False
    except FileNotFoundError:
        pass
    except ValueError:
        pass

  0%|          | 0/2000 [00:00<?, ?it/s]

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/alberto/anaconda3/envs/edc_mo_pdac_val/lib/python3.11/site-packages/pytorch_lightning/loops/utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
/home/alberto/anaconda3/envs/edc_mo_pdac_val/lib/python3.11/site-packages/pytorch_lightning/trainer/configuration_validator.py:72: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/alberto/anaconda3/envs/edc_mo_pdac_val/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` 

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

LR finder stopped early after 99 steps due to diverging loss.
Learning rate set to 0.0019054607179632484
Restoring states from the checkpoint path at /home/alberto/PycharmProjects/edc_mo_pdac_val/.lr_find_8be95940-ca4a-4df2-a29e-9aa924a42c61.ckpt
Restored all states from the checkpoint at /home/alberto/PycharmProjects/edc_mo_pdac_val/.lr_find_8be95940-ca4a-4df2-a29e-9aa924a42c61.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: tensorboard/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type       | Params
------------------------------------------
0  | encoder_0 | FCN        | 109 K 
1  | decoder_0 | FCN        | 972 K 
2  | encoder_1 | FCN        | 319 K 
3  | decoder_1 | FCN        | 1.7 M 
4  | encoder_2 | FCN        | 168 K 
5  | decoder_2 | FCN        | 1.2 M 
6  | encoder_3 | FCN        | 105 K 
7  | decoder_3 | FCN        |

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

...

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.025118864315095822
Restoring states from the checkpoint path at /home/alberto/PycharmProjects/edc_mo_pdac_val/.lr_find_b3bbe70f-341d-49f9-9c50-c4f7d73ae11e.ckpt
Restored all states from the checkpoint at /home/alberto/PycharmProjects/edc_mo_pdac_val/.lr_find_b3bbe70f-341d-49f9-9c50-c4f7d73ae11e.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type       | Params
------------------------------------------
0  | encoder_0 | FCN        | 75.3 K
1  | decoder_0 | FCN        | 91.5 K
2  | encoder_1 | FCN        | 218 K 
3  | decoder_1 | FCN        | 244 K 
4  | encoder_2 | FCN        | 114 K 
5  | decoder_2 | FCN        | 134 K 
6  | encoder_3 | FCN        | 72.3 K
7  | decoder_3 | FCN        | 88.2 K
8  | encoder_4 | FCN        | 63.7 K
9  | decoder_4 | FCN

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.07585775750291836
Restoring states from the checkpoint path at /home/alberto/PycharmProjects/edc_mo_pdac_val/.lr_find_41eb966a-7f32-4c43-94b6-9bdb1becf2b4.ckpt
Restored all states from the checkpoint at /home/alberto/PycharmProjects/edc_mo_pdac_val/.lr_find_41eb966a-7f32-4c43-94b6-9bdb1becf2b4.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type       | Params
------------------------------------------
0  | encoder_0 | FCN        | 75.3 K
1  | decoder_0 | FCN        | 91.5 K
2  | encoder_1 | FCN        | 218 K 
3  | decoder_1 | FCN        | 244 K 
4  | encoder_2 | FCN        | 114 K 
5  | decoder_2 | FCN        | 134 K 
6  | encoder_3 | FCN        | 72.3 K
7  | decoder_3 | FCN        | 88.2 K
8  | encoder_4 | FCN        | 63.7 K
9  | decoder_4 | FCN 

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 0.02089296130854041
Restoring states from the checkpoint path at /home/alberto/PycharmProjects/edc_mo_pdac_val/.lr_find_62b5b388-1f97-44f9-9a2d-6d59884d6bde.ckpt
Restored all states from the checkpoint at /home/alberto/PycharmProjects/edc_mo_pdac_val/.lr_find_62b5b388-1f97-44f9-9a2d-6d59884d6bde.ckpt
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name      | Type       | Params
------------------------------------------
0  | encoder_0 | FCN        | 75.3 K
1  | decoder_0 | FCN        | 91.5 K
2  | encoder_1 | FCN        | 218 K 
3  | decoder_1 | FCN        | 244 K 
4  | encoder_2 | FCN        | 114 K 
5  | decoder_2 | FCN        | 134 K 
6  | encoder_3 | FCN        | 72.3 K
7  | decoder_3 | FCN        | 88.2 K
8  | encoder_4 | FCN        | 63.7 K
9  | decoder_4 | FCN 

In [ ]:
optimization_study.best_params

In [ ]:
fig = optuna.visualization.plot_optimization_history(optimization_study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_param_importances(optimization_study)
fig.show()

In [ ]:
fig = optuna.visualization.plot_slice(optimization_study)
fig.show()

In [ ]:
optimization_results = pd.read_csv(os.path.join(settings.optimization_path, f"optimization_results_{date}.csv"))
best_trial = optimization_results.iloc[0]
print("optimization_results.shape", optimization_results.shape)
optimization_results.head()